# Calculating the value of turf zones using Pandas

In [1]:
%matplotlib inline
from IPython.display import display
import requests
import pandas as pd
import datetime
import numexpr as ne

In [2]:
url = "http://api.turfgame.com/v4/zones/all" # get request returns .json 
r = requests.get(url)
df = pd.read_json(r.content) # create a df containing all zone info
current_date = datetime.datetime.now() # system time at time of import
print(current_date)

2016-01-04 01:12:42.289691


In [3]:
display(df.head(),(df.dtypes))

,dateCreated,id,latitude,longitude,name,pointsPerHour,region,takeoverPoints,totalTakeovers
0,2014-06-15T20:02:37+0000,31571,55.649181,12.609056,Stenringen,1,"{'id': 172, 'country': 'dk', 'name': 'Hovedsta...",185,21
1,2013-06-15T08:00:00+0000,14639,55.947079,-3.206477,GrandSquare,1,"{'id': 200, 'country': 'gb', 'name': 'Scotland'}",185,91
2,2015-06-05T22:00:00+0000,54209,69.011607,23.044486,Kautokeino,1,"{'id': 183, 'country': 'no', 'name': 'Nordnorge'}",185,2
3,2013-06-15T08:00:00+0000,18661,53.754283,-1.526638,LanshawZone,0,"{'id': 202, 'country': 'gb', 'name': 'Yorkshir...",250,0
4,2013-06-15T08:00:00+0000,18958,54.593431,-5.953459,Hospitality,0,"{'id': 206, 'country': 'gb', 'name': 'Northern...",250,1


dateCreated        object
id                  int64
latitude          float64
longitude         float64
name               object
pointsPerHour       int64
region             object
takeoverPoints      int64
totalTakeovers      int64
dtype: object

In [4]:
region = pd.DataFrame(df.pop("region").tolist())
df = df.join(region, rsuffix="_region")

In [5]:
df = df[df["name_region"].str.contains("Uppsala")]
display(df.head())

,dateCreated,id,latitude,longitude,name,pointsPerHour,takeoverPoints,totalTakeovers,country,id_region,name_region
34,2013-07-16T18:53:48+0000,20828,59.793476,17.775389,MoraStenRast,5,125,1172,se,142,Uppsala
71,2015-03-22T11:00:00+0000,46898,59.799585,17.671343,Brasse,3,155,399,se,142,Uppsala
75,2013-02-08T21:42:53+0000,14797,59.570418,17.482116,BålWoods,1,185,725,se,142,Uppsala
124,2015-02-08T10:00:00+0000,44579,59.946952,17.545636,OutOfControl,2,170,356,se,142,Uppsala
130,2014-06-19T12:00:00+0000,31843,59.617637,17.077094,EnaAlle,5,125,1134,se,142,Uppsala


In [6]:
df.describe()

,id,latitude,longitude,pointsPerHour,takeoverPoints,totalTakeovers,id_region
count,1132.000000,1132.000000,1132.000000,1132.000000,1132.000000,1132.000000,1132
mean,29541.467314,59.873421,17.544106,3.275618,150.998233,1343.028269,142
std,17494.576374,0.233939,0.294931,2.174933,32.923726,1980.916162,0
min,341.000000,59.472589,16.792471,0.000000,65.000000,0.000000,142
25%,12408.750000,59.740261,17.428960,1.000000,125.000000,147.000000,142
50%,31360.000000,59.847449,17.612778,3.000000,155.000000,578.500000,142
75%,44797.250000,59.898972,17.672524,5.000000,185.000000,1782.250000,142
max,62429.000000,60.660998,19.133256,9.000000,250.000000,16000.000000,142


In [7]:
df.drop(['id', 'id_region', 'country', 'name_region'],inplace=True, axis = 1)

In [8]:
df['dateCreated'] = pd.to_datetime(df['dateCreated'])
display(df.head(), df.dtypes)

,dateCreated,latitude,longitude,name,pointsPerHour,takeoverPoints,totalTakeovers
34,2013-07-16 18:53:48,59.793476,17.775389,MoraStenRast,5,125,1172
71,2015-03-22 11:00:00,59.799585,17.671343,Brasse,3,155,399
75,2013-02-08 21:42:53,59.570418,17.482116,BålWoods,1,185,725
124,2015-02-08 10:00:00,59.946952,17.545636,OutOfControl,2,170,356
130,2014-06-19 12:00:00,59.617637,17.077094,EnaAlle,5,125,1134


dateCreated       datetime64[ns]
latitude                 float64
longitude                float64
name                      object
pointsPerHour              int64
takeoverPoints             int64
totalTakeovers             int64
dtype: object

## Calculations

In [9]:
df['time_elapsed'] = current_date - df['dateCreated']
display(df['time_elapsed'].head())

34     901 days 06:18:54.289691
71     287 days 14:12:42.289691
75    1059 days 03:29:49.289691
124    329 days 15:12:42.289691
130    563 days 13:12:42.289691
Name: time_elapsed, dtype: timedelta64[ns]

In [10]:
df['time_elapsed'] = df['time_elapsed'].astype('timedelta64[D]') * 24
display(df['time_elapsed'].head())

34     21624
71      6888
75     25416
124     7896
130    13512
Name: time_elapsed, dtype: float64

In [11]:
df['hold_time'] = df['time_elapsed'] / df['totalTakeovers']
df['total_value'] = (df['hold_time'] * df['pointsPerHour']) + df['takeoverPoints']

In [12]:
display(df.columns, df.head())

Index(['dateCreated', 'latitude', 'longitude', 'name', 'pointsPerHour',
       'takeoverPoints', 'totalTakeovers', 'time_elapsed', 'hold_time',
       'total_value'],
      dtype='object')

,dateCreated,latitude,longitude,name,pointsPerHour,takeoverPoints,totalTakeovers,time_elapsed,hold_time,total_value
34,2013-07-16 18:53:48,59.793476,17.775389,MoraStenRast,5,125,1172,21624,18.450512,217.252560
71,2015-03-22 11:00:00,59.799585,17.671343,Brasse,3,155,399,6888,17.263158,206.789474
75,2013-02-08 21:42:53,59.570418,17.482116,BålWoods,1,185,725,25416,35.056552,220.056552
124,2015-02-08 10:00:00,59.946952,17.545636,OutOfControl,2,170,356,7896,22.179775,214.359551
130,2014-06-19 12:00:00,59.617637,17.077094,EnaAlle,5,125,1134,13512,11.915344,184.576720


In [13]:
df.sort_values('total_value', ascending=False, inplace=True)
display(df.head(n=10))

,dateCreated,latitude,longitude,name,pointsPerHour,takeoverPoints,totalTakeovers,time_elapsed,hold_time,total_value
29754,2014-07-28 22:00:00,59.551758,17.552917,BålstaDopp,1,185,4,12576,3144.000000,3329.000000
18923,2015-04-22 19:15:00,60.183703,18.169321,Plurret,1,185,3,6144,2048.000000,2233.000000
9301,2014-09-21 10:00:00,60.300895,19.133256,Märket,1,185,6,11256,1876.000000,2061.000000
8374,2015-02-05 19:31:54,60.015222,17.106797,JobbaFörZonen,1,185,5,7968,1593.600000,1778.600000
20843,2015-05-07 20:55:00,59.971086,17.088223,Upplandsberg,1,185,4,5784,1446.000000,1631.000000
20675,2015-01-12 17:00:00,59.591547,17.068185,AquanautZone,1,185,9,8544,949.333333,1134.333333
515,2015-07-26 08:23:30,59.745146,17.624096,Flässjan,2,170,9,3864,429.333333,1028.666667
3139,2014-07-28 22:00:00,60.628049,17.416366,Skutbad,1,185,15,12576,838.400000,1023.400000
22719,2015-02-14 05:00:00,59.726284,17.544565,Arnöhuvud,1,185,10,7752,775.200000,960.200000
23373,2015-02-14 05:00:00,59.737383,17.566068,Koffsan,1,185,10,7752,775.200000,960.200000


In [14]:
df = df.query('latitude > 59.787105 & latitude < 59.887502 & \
              longitude < 17.677434 & longitude > 17.630056')

# Data visualisation: plotting to a Google map using gmplot

In [15]:
import numpy as np
from gmplot import GoogleMapPlotter

NB: At the time of writing, gmplot is only available for Python 2.x. However 2to3 can be used to port the code to Python 3.x.

We'll use the coordinates for the first turf zone in our list as a point on which to centre our map.

In [16]:
centre_lat = df.iat[0, 1] #df.iat[row, col]
centre_lon = df.iat[0, 2]

We can use pd.iat for indexing and create to lists to store our coordinates.

In [17]:
number_of_zones = 20 # Number of zones to plot
lat = []
lon = []

for zone in range(number_of_zones):
    lat.append(df.iat[zone, 1])
    lon.append(df.iat[zone, 2])

Finally pass the lists of latitudes and longitudes to a GoogleMapPlotter object for plotting and save the file.

In [18]:
gmap = GoogleMapPlotter(centre_lat, centre_lon, 12) #centre lat, lon, map size
gmap.scatter(lat,lon)
gmap.draw("TopTwentyZones.html")